### Install Library

In [ ]:
from IPython.display import clear_output
!pip uninstall -y numpy
!pip cache purge
!pip install numpy==1.26.4
clear_output()
print("Numpy install successful!")

import os
import IPython
os._exit(0)

In [1]:
from IPython.display import clear_output

!pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0
!pip install dgl -f https://data.dgl.ai/wheels/torch-2.2/repo.html
!pip install torchmetrics==1.2.1 transformers==4.38.0
!pip install safetensors==0.4.1
!pip install torcheval
!pip install scikit-learn
!pip install deep-translator
clear_output()

import os
import dgl
import torch
import torchmetrics
import transformers
import torcheval

os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"
device = torch.device("cpu")

try:
    import dgl
    import dgl.graphbolt as gb
    installed = True
except ImportError as error:
    installed = False
    print(error)

print("DGL installed!" if installed else "DGL not found!")
print("PyTorch Version: ", torch.__version__)
print("TorchMetrics Version: ", torchmetrics.__version__)
print("Transformers Version: ", transformers.__version__)
print("DGL Version: ", dgl.__version__)
print("TorchEval Is: ", torcheval.__version__)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
DGL installed!
PyTorch Version:  2.2.0+cu121
TorchMetrics Version:  1.2.1
Transformers Version:  4.38.0
DGL Version:  2.4.0
TorchEval Is:  0.0.7


### Import Dataset

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive')

ILP_Date_Zip_File = '/content/drive/MyDrive/DataSet/ILPDataSet.zip'
!unzip -q {ILP_Date_Zip_File} -d {'/content'}

datasets = sorted([folder for folder in os.listdir('/content') if os.path.isdir(os.path.join('/content', folder))])
def create_dataset_dict(base_dir:str='/content'):
    datasets = {}
    for dataset_name in os.listdir(base_dir):
        dataset_path = os.path.join(base_dir, dataset_name)
        if os.path.isdir(dataset_path):
            datasets[dataset_name] = {
                "train": os.path.join(dataset_path, "train.txt"),
                "valid": os.path.join(dataset_path, "valid.txt"),
                "test":  os.path.join(dataset_path, "test.txt")}
    return datasets

# Save Path Dictionay
ILP_dataset_paths = create_dataset_dict('/content/ILPDataSet')
ILP_dataset_paths = dict(sorted(ILP_dataset_paths.items()))

Mounted at /content/drive


### Analysis PersianILP With English BencmarkDataset

In [3]:
import os
import pandas as pd
import networkx as nx
from collections import Counter
from tabulate import tabulate

def load_data(file_path):
    sep = "," if file_path.endswith('.csv') else "\t"
    return pd.read_csv(file_path, sep=sep, header=None, names=["head", "relation", "tail"])

def analyze_graph_metrics(file_path):
    df = load_data(file_path)
    G = nx.MultiDiGraph()
    G.add_edges_from(zip(df["head"], df["tail"], df["relation"]))

    degrees = dict(G.degree())
    counter = Counter(degrees.values())
    avg_deg = sum(degrees.values()) / G.number_of_nodes() if G.number_of_nodes() else 0

    # محاسبه‌ی تعداد سه‌تایی‌ها، موجودیت‌ها و روابط
    num_triples = len(df)
    num_entities = len(set(df["head"]).union(set(df["tail"])))
    num_relations = len(set(df["relation"]))

    return {
        "Triples": num_triples,
        "Entities": num_entities,
        "Relations": num_relations,
        "Deg_1": counter.get(1, 0),
        "Deg_2": counter.get(2, 0),
        "Deg_3": counter.get(3, 0),
        "Avg_Degree": round(avg_deg, 2),
        "Density": round(nx.density(G), 6),
        "Sparsity": round(1 - nx.density(G), 6)
    }

def process_file(file_path, label):
    if os.path.isfile(file_path) and file_path.endswith(('.csv', '.txt')):
        metrics = analyze_graph_metrics(file_path)
        if metrics:
            metrics['Dataset'] = label
            return metrics
    return None

def analyze_all_datasets(all_dirs):
    results = []
    for base_dir in all_dirs:
        for root, _, files in os.walk(base_dir):
            dataset_name = os.path.basename(root)
            for file in files:
                path = os.path.join(root, file)
                ext = os.path.splitext(file)[1].lower()
                label_type = "CSV" if ext == '.csv' else "TXT"
                label = f"{dataset_name}_{os.path.splitext(file)[0]}"
                result = process_file(path, label)
                if result:
                    results.append(result)

    return pd.DataFrame(results)[[
        "Dataset", "Triples", "Entities", "Relations",
        "Deg_1", "Deg_2", "Deg_3", "Avg_Degree", "Density", "Sparsity"
    ]]

# مسیر دیتاست‌ها را مشخص کن
all_dirs = [
    "/content/ILPDataSet",
    "/content/PersianILP-trainTest"
]

# اجرای تحلیل و نمایش جدول
df_result = analyze_all_datasets(all_dirs).sort_values("Dataset")
print(tabulate(df_result, headers="keys", tablefmt="grid", showindex=False))

+---------------------+-----------+------------+-------------+---------+---------+---------+--------------+-----------+------------+
| Dataset             |   Triples |   Entities |   Relations |   Deg_1 |   Deg_2 |   Deg_3 |   Avg_Degree |   Density |   Sparsity |
+=====================+===========+============+=============+=========+=========+=========+==============+===========+============+
| PersianILP-V1_test  |      3000 |       2869 |         513 |    1757 |     640 |     213 |         2.09 |  0.000364 |   0.999636 |
+---------------------+-----------+------------+-------------+---------+---------+---------+--------------+-----------+------------+
| PersianILP-V1_train |     10500 |      11029 |         966 |    7407 |    2000 |     736 |         1.9  |  8.6e-05  |   0.999914 |
+---------------------+-----------+------------+-------------+---------+---------+---------+--------------+-----------+------------+
| PersianILP-V1_valid |      1500 |       1241 |         355 |     63

### Inductive Link Prediction

In [6]:
import os
import pandas as pd
from tabulate import tabulate

def analyze_kg_files(base_dir):
    """Analyze knowledge graph files with better error handling"""
    results = []

    for dataset in sorted(os.listdir(base_dir)):
        dataset_path = os.path.join(base_dir, dataset)
        if not os.path.isdir(dataset_path):
            continue

        stats = {'Dataset': dataset}

        for split in ['train', 'test']:
            for ext in ['.csv', '.txt']:
                file_path = os.path.join(dataset_path, f"{split}{ext}")
                if not os.path.exists(file_path):
                    continue

                try:
                    # Read file with automatic format detection
                    try:
                        # First, try reading with header
                        df = pd.read_csv(file_path)
                        # If required columns are missing, read without header
                        if not all(col in df.columns for col in ['head', 'relation', 'tail']):
                            df = pd.read_csv(file_path, sep='\t' if ext == '.txt' else ',',
                                             header=None, names=['head', 'relation', 'tail'])
                    except:
                        # If error occurs, read without header
                        df = pd.read_csv(file_path, sep='\t' if ext == '.txt' else ',',
                                         header=None, names=['head', 'relation', 'tail'])

                    # Compute basic statistics
                    stats.update({
                        f'{split}_triples': int(len(df)),
                        f'{split}_relations': int(df['relation'].nunique()),
                        f'{split}_entities': int(pd.concat([df['head'], df['tail']]).nunique())
                    })
                    break

                except Exception as e:
                    print(f"Error processing {file_path}: {str(e)}")
                    continue

        results.append(stats)

    return pd.DataFrame(results)

def show_stats(base_dir):
    """Display results table with proper formatting"""
    df = analyze_kg_files(base_dir)

    if df.empty:
        print("No valid dataset found")
        return

    # Select and sort columns
    columns = [
        'Dataset',
        'train_triples', 'test_triples',
        'train_relations', 'test_relations',
        'train_entities', 'test_entities'
    ]

    # Drop rows with NaN values
    df = df.dropna(subset=['train_triples'])[columns].sort_values('Dataset')

    # Rename columns
    df.columns = [
        'Dataset',
        'train_triples', 'test_triples',
        'train_relations', 'test_relations',
        'train_entities', 'test_entities'
    ]

    # Display numbers as integers
    print(tabulate(
        df,
        headers='keys',
        tablefmt='grid',
        showindex=False,
        numalign="right",
        floatfmt=".0f"
    ))

# Sample execution
show_stats("/content/ILPDataSet")

+---------------+-----------------+----------------+-------------------+------------------+------------------+-----------------+
| Dataset       |   train_triples |   test_triples |   train_relations |   test_relations |   train_entities |   test_entities |
+===============+=================+================+===================+==================+==================+=================+
| PersianILP-V1 |           10500 |           3000 |               966 |              513 |            11029 |            2869 |
+---------------+-----------------+----------------+-------------------+------------------+------------------+-----------------+
| PersianILP-V2 |           10500 |           3000 |               966 |              554 |            11029 |            4245 |
+---------------+-----------------+----------------+-------------------+------------------+------------------+-----------------+
| PersianILP-V3 |           10500 |           3000 |               984 |              541 |      

### Create DGL Dataset

In [7]:
import dgl
import torch
import pandas as pd
from dgl.data import DGLDataset

class PersianDGLDataset(DGLDataset):
    def __init__(self, train_file, test_file, seed=42):
        self.train_file = train_file
        self.test_file = test_file
        self.seed = seed
        self.process()
        super().__init__(name="PersianLinkPrediction")

    def process(self):
        # Initialize mappings
        self.entity2id = {}
        self.relation2id = {}
        ent_id, rel_id = 0, 0

        # Process training data
        train_triples = self._load_and_process_file(self.train_file, ent_id, rel_id)
        ent_id, rel_id = len(self.entity2id), len(self.relation2id)

        # Process test data (using same mappings)
        test_triples = self._load_and_process_file(self.test_file, ent_id, rel_id)

        # Build graphs
        self.graphs = {
            "train": self._build_graph(train_triples),
            "test": self._build_graph(test_triples)
        }

    def _load_file(self, file_path):
        """Load file based on its extension"""
        if file_path.endswith('.csv'):
            return pd.read_csv(file_path)
        elif file_path.endswith('.txt'):
            return pd.read_csv(file_path, sep='\t', header=None,
                             names=['subjectLabel', 'predicateLabel', 'objectLabel'])
        else:
            raise ValueError("Unsupported file format. Only .csv and .txt files are supported.")

    def _load_and_process_file(self, file_path, ent_id_start, rel_id_start):
        """Load and process a single file, updating mappings"""
        triples = []
        df = self._load_file(file_path)

        for _, row in df.iterrows():
            h, r, t = row['subjectLabel'], row['predicateLabel'], row['objectLabel']

            # Update entity mappings
            for ent in [h, t]:
                if ent not in self.entity2id:
                    self.entity2id[ent] = ent_id_start
                    ent_id_start += 1

            # Update relation mappings
            if r not in self.relation2id:
                self.relation2id[r] = rel_id_start
                rel_id_start += 1

            triples.append((
                self.entity2id[h],
                self.relation2id[r],
                self.entity2id[t]))

        return triples

    def _build_graph(self, triples):
        """Build DGL graph from triples"""
        src, rel, dst = zip(*triples)
        src = torch.tensor(src)
        dst = torch.tensor(dst)
        rel = torch.tensor(rel)

        g = dgl.graph((src, dst), num_nodes=len(self.entity2id))
        g.edata["e_type"] = rel
        g.edata["edge_mask"] = torch.ones(g.num_edges(), dtype=torch.bool)
        g.ndata["ntype"] = torch.zeros(g.num_nodes(), dtype=torch.int)
        g.ndata["feat"] = torch.randn(g.num_nodes(), 64)
        return g

    def __getitem__(self, split):
        return self.graphs[split]

    def __len__(self):
        return len(self.graphs)

class GraphBatchDataset(torch.utils.data.Dataset):
    def __init__(self, graphs, pos_graphs, neg_graphs):
        self.graphs = graphs
        self.pos_graphs = pos_graphs
        self.neg_graphs = neg_graphs

    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return {
            "graph": self.graphs[idx],
            "pos_graph": self.pos_graphs[idx],
            "neg_graph": self.neg_graphs[idx]}


dataset = PersianDGLDataset(train_file = ILP_dataset_paths['PersianILP-V1']['train'],
                            test_file = ILP_dataset_paths['PersianILP-V1']['test'])
train_g = dataset["train"]
test_g = dataset["test"]

### Generate Positive Graph And Negative Graph

In [8]:
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import dgl
import scipy.sparse as sp
from tabulate import tabulate
import torch

class GraphNegativeSampler:
    def __init__(self, train_graph, test_graph, train_neg_ratio=1.0, test_neg_ratio=1.0):
        self.train_graph = train_graph
        self.test_graph = test_graph
        self.train_neg_ratio = train_neg_ratio
        self.test_neg_ratio = test_neg_ratio
        self.train_pos_g, self.train_neg_g = self._prepare_graphs(train_graph, train_neg_ratio)
        self.test_pos_g, self.test_neg_g = self._prepare_graphs(test_graph, test_neg_ratio)

    def _generate_negative_samples(self, graph):
        u, v = graph.edges()
        adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())),
                          shape=(graph.num_nodes(), graph.num_nodes()))
        return np.where(1 - adj.todense() - np.eye(graph.num_nodes()) != 0)

    def _prepare_graphs(self, graph, ratio):
        return ( self._create_positive_graph(graph),
                 self._create_negative_graph(graph, ratio))

    def _create_positive_graph(self, graph):
        g = dgl.graph(graph.edges(), num_nodes=graph.num_nodes())
        g.edata["e_type"] = graph.edata["e_type"]
        g.ndata.update({k: graph.ndata[k] for k in ["feat", "ntype"]})
        return g

    def _create_negative_graph(self, graph, ratio):
        neg_u, neg_v = self._generate_negative_samples(graph)
        num_samples = int(graph.num_edges() * ratio)
        replace = len(neg_u) < num_samples
        sample_ids = np.random.choice(len(neg_u), num_samples, replace=replace)

        g = dgl.graph((neg_u[sample_ids], neg_v[sample_ids]), num_nodes=graph.num_nodes())
        g.edata["e_type"] = torch.randint(0, graph.edata["e_type"].max().item()+1, (g.num_edges(),))
        g.ndata.update({
            "feat": graph.ndata["feat"],
            "ntype": torch.ones(graph.num_nodes(), dtype=torch.int)})
        return g

    @property
    def training_graphs(self):
        return self.train_pos_g, self.train_neg_g

    @property
    def test_graphs(self):
        return self.test_pos_g, self.test_neg_g

# Sampling From Knowladge Graph
sampler = GraphNegativeSampler(dataset['train'],
                               dataset['test'],
                               train_neg_ratio=1,
                               test_neg_ratio=1)

train_pos, train_neg = sampler.training_graphs
test_pos, test_neg = sampler.test_graphs

### Link Prediction Model

In [9]:
from dgl.nn import SAGEConv
import torch.nn as nn

class ImprovedGraphSAGE(nn.Module):
  def __init__(self, in_feats, h_feats, out_feats, dropout=0.5):
        super(ImprovedGraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, "mean")
        self.conv2 = SAGEConv(h_feats, out_feats, "mean")
        self.dropout = nn.Dropout(dropout)

  def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.dropout(h)
        h = self.conv2(g, h)
        return h


import dgl.function as fn
class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata["h"] = h
            g.apply_edges(fn.u_dot_v("h", "h", "score"))
            return g.edata["score"][:, 0]

### Train method

In [10]:
from torch.utils.data import DataLoader
import itertools
from tqdm import tqdm
import dgl

def train_model(model,
                pred,
                dataloader,
                epochs,
                lr=0.01):

    optimizer = torch.optim.Adam(itertools.chain(model.parameters(),
                                                 pred.parameters()),
                                                 lr=lr)

    all_losses = []
    for epoch in tqdm(range(epochs)):
        epoch_loss = 0.0

        for batch in dataloader:
            batch_graph = batch["graph"]    # گراف اصلی
            pos_graph = batch["pos_graph"]  # گراف مثبت
            neg_graph = batch["neg_graph"]  # گراف منفی

            # Forward pass
            h = model(batch_graph, batch_graph.ndata["feat"])
            pos_score = pred(pos_graph, h)
            neg_score = pred(neg_graph, h)
            loss = compute_loss(pos_score,neg_score)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            e = epoch
            loss = epoch_loss

        all_losses.append(epoch_loss)

    print(f"\nEpoch: {e}, Loss: {loss:.4f}")
    return h, all_losses

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

### Train And Evaluation

In [11]:
import torch
import torch.nn.functional as F
import numpy as np
import itertools
from tqdm import tqdm
from dgl.dataloading import GraphDataLoader
from IPython.display import clear_output
from sklearn.metrics import average_precision_score, roc_auc_score
from tabulate import tabulate
from torchmetrics.retrieval import RetrievalMRR, RetrievalHitRate
from sklearn import metrics

def train_and_evaluate_model(result:dict,
                             dataset_name,
                             ILP_dataset_paths,
                             h_feats=16,
                             out_feats=10,
                             dropout=0.5,
                             epochs=2000,
                             lr=0.001,
                             train_neg_ratio=10,
                             test_neg_ratio=1):

    # ===== Step 1: Dataset Preparation =====
    graphs = PersianDGLDataset(
        train_file=ILP_dataset_paths[dataset_name]['train'],
        test_file=ILP_dataset_paths[dataset_name]['test']
    )

    sampler = GraphNegativeSampler(
        graphs['train'], graphs['test'],
        train_neg_ratio=train_neg_ratio,
        test_neg_ratio=test_neg_ratio
    )

    train_pos_g, train_neg_g = sampler.training_graphs
    test_pos_g, test_neg_g = sampler.test_graphs

    train_dataset = GraphBatchDataset([graphs['train']], [train_pos_g], [train_neg_g])
    train_loader = GraphDataLoader(train_dataset, batch_size=1, collate_fn=lambda x: x[0])

    test_dataset = GraphBatchDataset([graphs['test']], [test_pos_g], [test_neg_g])
    test_loader = GraphDataLoader(test_dataset, batch_size=1, collate_fn=lambda x: x[0])

    # ===== Step 2: Training =====
    def compute_loss(pos_score, neg_score):
        scores = torch.cat([pos_score, neg_score])
        labels = torch.cat([
            torch.ones(pos_score.shape[0]),
            torch.zeros(neg_score.shape[0])
        ])
        return F.binary_cross_entropy_with_logits(scores, labels)

    in_feats = graphs['train'].ndata['feat'].shape[1]
    model = ImprovedGraphSAGE(
        in_feats=in_feats,
        h_feats=h_feats,
        out_feats=out_feats,
        dropout=dropout
    )
    pred = DotPredictor()

    optimizer = torch.optim.Adam(
        itertools.chain(model.parameters(), pred.parameters()),
        lr=lr
    )

    for epoch in tqdm(range(epochs)):
        for batch in train_loader:
            h = model(batch['graph'], batch['graph'].ndata['feat'])
            pos_score = pred(batch['pos_graph'], h)
            neg_score = pred(batch['neg_graph'], h)
            loss = compute_loss(pos_score, neg_score)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # ===== Step 3: Evaluation =====
    pos_scores, pos_labels = [], []
    neg_scores, neg_labels = [], []
    hit1_list, hit3_list, hit10_list = [], [], []

    with torch.no_grad():
        ranks = []
        for batch in test_loader:
            h = model(batch['graph'], batch['graph'].ndata['feat'])
            score_pos = pred(batch['pos_graph'], h).squeeze()
            score_neg = pred(batch['neg_graph'], h).squeeze()

            neg_per_pos = len(score_neg) // len(score_pos)
            pos_scores += score_pos.tolist() if score_pos.dim() > 0 else [score_pos.item()]
            neg_scores += score_neg.tolist() if score_neg.dim() > 0 else [score_neg.item()]
            pos_labels += [1] * len(score_pos) if score_pos.dim() > 0 else [1]
            neg_labels += [0] * len(score_neg) if score_neg.dim() > 0 else [0]

            score_pos_exp = score_pos.view(-1, 1).repeat(1, neg_per_pos).view(-1)
            scores = torch.stack([score_pos_exp, score_neg], dim=1)
            scores = torch.softmax(scores, dim=1).cpu().numpy()
            rank = np.argwhere(np.argsort(scores, axis=1)[:, ::-1] == 0)[:, 1] + 1

            ranks += rank.tolist()
            hit1_list += [1 if r <= 1 else 0 for r in rank]
            hit3_list += [1 if r <= 3 else 0 for r in rank]
            hit10_list += [1 if r <= 10 else 0 for r in rank]

    # ===== Step 4: Result Metrics =====
    result[dataset_name] = {
        "AUC": metrics.roc_auc_score(pos_labels + neg_labels, pos_scores + neg_scores),
        "AUC_PR": metrics.average_precision_score(pos_labels + neg_labels, pos_scores + neg_scores),
        "MRR": np.mean(1.0 / np.array(ranks)).item(),
        "Hit1": np.mean(hit1_list),
        "Hit3": np.mean(hit3_list),
        "Hit10": np.mean(hit10_list)}

    return result

from IPython.display import clear_output
from tabulate import tabulate
def display_results_table(result_dict):
    clear_output()
    headers = ['Dataset', 'AUC', 'AUC_PR', 'MRR', 'Hit1', 'Hit3', 'Hit10']
    rows = []
    for name, metrics in result_dict.items():
        row = [name] + [metrics[h] for h in headers[1:]]
        rows.append(row)
    print("\n" + tabulate(rows,
                          headers=headers,
                          tablefmt="fancy_grid",
                          floatfmt=".4f"))

**Experiment with 1 Negative Samples**

In [ ]:
import numpy as np
from collections import defaultdict
from tabulate import tabulate

all_results = defaultdict(list)
num_runs = 20

for run in range(num_runs):
    clear_output()
    print(f"\nRun {run + 1}/{num_runs}")
    result = {}

    for name, path in ILP_dataset_paths.items():
            result = train_and_evaluate_model(
            result,
            name,
            ILP_dataset_paths,
            h_feats=32,
            out_feats=8,
            dropout=0.5,
            epochs=2000,
            lr=0.001,
            train_neg_ratio=1,
            test_neg_ratio=1)

    # Store results for this run
    for dataset_name, result_metrics in result.items():
        all_results[dataset_name].append(result_metrics)


final_results = {}
for dataset_name, runs in all_results.items():
    result_metrics = runs[0].keys()  # Get metric names
    dataset_stats = {}
    for metric in result_metrics:
        values = [run[metric] for run in runs]
        dataset_stats[f"{metric}_mean"] = np.mean(values)
        dataset_stats[f"{metric}_std"] = np.std(values)

    final_results[dataset_name] = dataset_stats

# Display final results
clear_output()
headers = [
    'Dataset',
    'AUC (mean±std)',
    'AUC_PR (mean±std)',
    'MRR (mean±std)',
    'Hit1 (mean±std)',
    'Hit3 (mean±std)',
    'Hit10 (mean±std)'
]

rows = []
for name, result_metrics in final_results.items():
    row = [name]
    for metric in ['AUC', 'AUC_PR', 'MRR', 'Hit1', 'Hit3', 'Hit10']:
        mean = result_metrics[f"{metric}_mean"]
        std = result_metrics[f"{metric}_std"]
        row.append(f"{mean:.4f}±{std:.4f}")
    rows.append(row)

print("\nFinal Results After 20 Runs:")
print(tabulate(rows, headers=headers, tablefmt="fancy_grid"))


Final Results After 20 Runs:
╒═══════════════╤══════════════════╤═════════════════════╤══════════════════╤═══════════════════╤═══════════════════╤════════════════════╕
│ Dataset       │ AUC (mean±std)   │ AUC_PR (mean±std)   │ MRR (mean±std)   │ Hit1 (mean±std)   │ Hit3 (mean±std)   │ Hit10 (mean±std)   │
╞═══════════════╪══════════════════╪═════════════════════╪══════════════════╪═══════════════════╪═══════════════════╪════════════════════╡
│ PersianILP-V1 │ 0.7535±0.0112    │ 0.7940±0.0100       │ 0.8766±0.0068    │ 0.7532±0.0135     │ 1.0000±0.0000     │ 1.0000±0.0000      │
├───────────────┼──────────────────┼─────────────────────┼──────────────────┼───────────────────┼───────────────────┼────────────────────┤
│ PersianILP-V2 │ 0.7928±0.0082    │ 0.8196±0.0078       │ 0.8963±0.0043    │ 0.7925±0.0086     │ 1.0000±0.0000     │ 1.0000±0.0000      │
├───────────────┼──────────────────┼─────────────────────┼──────────────────┼───────────────────┼───────────────────┼───────────────────

**Experiment with 50 Negative Samples**

In [12]:
import numpy as np
from collections import defaultdict
from tabulate import tabulate

all_results = defaultdict(list)
num_runs = 20

for run in range(num_runs):
    clear_output()
    print(f"\nRun {run + 1}/{num_runs}")
    result = {}

    for name, path in ILP_dataset_paths.items():
        result = train_and_evaluate_model(
            result,
            name,
            ILP_dataset_paths,
            h_feats=32,
            out_feats=8,
            dropout=0.5,
            epochs=2000,
            lr=0.001,
            train_neg_ratio=10,
            test_neg_ratio=50)

    # Store results for this run
    for dataset_name, result_metrics in result.items():
        all_results[dataset_name].append(result_metrics)


final_results = {}
for dataset_name, runs in all_results.items():
    result_metrics = runs[0].keys()  # Get metric names
    dataset_stats = {}
    for metric in result_metrics:
        values = [run[metric] for run in runs]
        dataset_stats[f"{metric}_mean"] = np.mean(values)
        dataset_stats[f"{metric}_std"] = np.std(values)

    final_results[dataset_name] = dataset_stats

# Display final results
clear_output()
headers = [
    'Dataset',
    'AUC (mean±std)',
    'AUC_PR (mean±std)',
    'MRR (mean±std)',
    'Hit1 (mean±std)',
    'Hit3 (mean±std)',
    'Hit10 (mean±std)'
]

rows = []
for name, result_metrics in final_results.items():
    row = [name]
    for metric in ['AUC', 'AUC_PR', 'MRR', 'Hit1', 'Hit3', 'Hit10']:
        mean = result_metrics[f"{metric}_mean"]
        std = result_metrics[f"{metric}_std"]
        row.append(f"{mean:.4f}±{std:.4f}")
    rows.append(row)

print("\nFinal Results After 20 Runs:")
print(tabulate(rows, headers=headers, tablefmt="fancy_grid"))


Final Results After 20 Runs:
╒═══════════════╤══════════════════╤═════════════════════╤══════════════════╤═══════════════════╤═══════════════════╤════════════════════╕
│ Dataset       │ AUC (mean±std)   │ AUC_PR (mean±std)   │ MRR (mean±std)   │ Hit1 (mean±std)   │ Hit3 (mean±std)   │ Hit10 (mean±std)   │
╞═══════════════╪══════════════════╪═════════════════════╪══════════════════╪═══════════════════╪═══════════════════╪════════════════════╡
│ PersianILP-V1 │ 0.6017±0.0184    │ 0.0719±0.0109       │ 0.8008±0.0093    │ 0.6016±0.0186     │ 1.0000±0.0000     │ 1.0000±0.0000      │
├───────────────┼──────────────────┼─────────────────────┼──────────────────┼───────────────────┼───────────────────┼────────────────────┤
│ PersianILP-V2 │ 0.6271±0.0253    │ 0.0616±0.0117       │ 0.8136±0.0127    │ 0.6272±0.0254     │ 1.0000±0.0000     │ 1.0000±0.0000      │
├───────────────┼──────────────────┼─────────────────────┼──────────────────┼───────────────────┼───────────────────┼───────────────────